In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

## Generate dataframe

In [14]:
class VisualOutcomes:
    
    def vision_list(self, df):
        """
        Returns a list of visual acuity for patient.
        Input: patient Pandas dataframe.
        Output: list.
        """
        df['CreatedDate'] = pd.to_datetime(df['CreatedDate'])
        df.sort_values(by=['CreatedDate'], inplace=True)
        lst = df['visual_acuity'].dropna()
        return lst.to_list()
    
    def mean_vision(self, df):
        """
        Returns the mean vision of a patient.
        Input: patient Pandas dataframe.
        Output: float (LogMAR letters).
        """
        lst = self.vision_list(df)
        return np.mean(lst)

    def loss_from_peak(self, df):
        """
        Returns the VLP for a patient.
        Vision Loss from Peak (VLP) is defined as max vision minus last vision.
        Input: Pandas dataframe.
        Output: float (LogMAR letters).
        """
        visions = self.vision_list(df)
        return max(visions) - visions[-1]
    
    def overall_visual_change(self, df):
        """
        Returns the OVC for a patient.
        Overall Visual Change (OVC) is defined as last vision minus first vision.
        Input: Pandas dataframe.
        Output: float (LogMAR letters).
        """
        visions = self.vision_list(df)
        last = (visions[-1] + visions[-2] + visions[-3]) / 3
        first = visions[0]
        return last - first
    
    def peak_visual_improvement(self, df):
        """
        Returns the PVI for a patient.
        Peak Visual Improvement (PVI) is defined as max vision minus initial vision.
        Input: Pandas dataframe.
        Output: float (LogMAR letters).
        """
        visions = self.vision_list(df)
        return max(visions) - visions[0]

    def proportion_above_baseline(self, df):
        """
        Returns the proportion of time above starting vision.
        Input: patient Pandas dataframe.
        Output: float (percentage).
        """
        lst = self.vision_list(df)
        starting_vision = lst[0]
        above_lst = [i for i in lst if i > starting_vision]
        if len(above_lst) != 0:
            mean = sum(above_lst) / len(above_lst)
        else:
            mean = 0
        return mean

    def patient_clean(self, df, number_years):
        """
        Shortens a patient's dataframe to x years after initiation.
        Input: patient Pandas dataframe, integer.
        Output: Pandas dataframe.
        """
        dates = df['CreatedDate'].to_list()
        first = dates[0]
        cutoff = first.replace(year = first.year + number_years)
        df = df[df['CreatedDate'] < cutoff]
        return df

    def time_above_baseline(self, df, number_years):
        """
        Returns the number of days a patient spent above baseline in first x years.
        Input: Pandas dataframe, integer.
        Output: integer (days).
        """
        df["CreatedDate"] = pd.to_datetime(df["CreatedDate"])
        df.sort_values(by=['CreatedDate'], inplace=True)
        dates = df["CreatedDate"].to_list()
        if (dates[-1] - dates[0]).days > (number_years * 365):
            df = patient_clean(df, number_years)
            vision = df['visual_acuity'].to_list()
            dates2 = df["CreatedDate"].to_list()
            days = 0
            starting_vision = vision[0]
            for i in range(1, len(vision)):
                if vision[i] > vision[0]:
                    between = (dates2[i] - dates2[i-1]).days
                    days += between
            return days
        else:
            return 'nil'
        
    def time_above_baseline2(self, df):
        """
        Returns the number of days a patient spent above baseline in first x years.
        Input: Pandas dataframe, integer.
        Output: integer (days).
        """
        df["CreatedDate"] = pd.to_datetime(df["CreatedDate"])
        df.sort_values(by=['CreatedDate'], inplace=True)
        dates = df["CreatedDate"].to_list()
        vision = df['visual_acuity'].to_list()
        dates2 = df["CreatedDate"].to_list()
        days = 0
        starting_vision = vision[0]
        for i in range(1, len(vision)):
            if vision[i] > vision[0]:
                between = (dates2[i] - dates2[i-1]).days
                days += between
        return days

    def time_to_peak(self, df):
        """
        Returns the TPVI for a patient.
        Time to Peak Visual Improvement (TPVI) is defined in days.
        Input: Pandas dataframe.
        Output: integer (days).
        """
        df["CreatedDate"] = pd.to_datetime(df["CreatedDate"])
        df.sort_values(by=['CreatedDate'], inplace=True)
        dates = df["CreatedDate"].to_list()
        max_value = df['visual_acuity'].max()
        df_fin = df[df['visual_acuity'] == max_value]
        initial_date = dates[0]
        final_date = df_fin.CreatedDate.iloc[0]
        return (final_date - initial_date).days
    
    def baseline_vision(self, df):
        """
        Returns the baseline vision for a patient.
        Input: Pandas dataframe.
        Output: integer (LogMAR letters).
        """
        lst = self.vision_list(df)
        return lst[0]

In [17]:
class Dataframe(VisualOutcomes):
    
    def get_df(self):
        """
        Returns the dataframe to be analysed (all visits).
        """
        df = pd.read_csv('/home/jupyter/charliemacuject/pharma_reports/data/dme.csv')
        df.drop(columns=['Unnamed: 0'], inplace=True)
        return df
    
    def dataframe_gen(self, pdf, pat_id):
        """
        Returns a dataframe of all adherence measures and visual outcomes.
        For a singular patient only (will be one row).
        Input: integer (patient id).
        Output: Pandas dataframe.
        """
        df = pdf[pdf["id"] == pat_id]
        data = {'mean_vision': [VisualOutcomes.mean_vision(self, df)], 
                'time_above_baseline': [VisualOutcomes.time_above_baseline2(self, df)],
                'peak_visual_improvement': [VisualOutcomes.peak_visual_improvement(self, df)],
                'overall_visual_change': [VisualOutcomes.overall_visual_change(self, df)],
                'time_to_peak': [VisualOutcomes.time_to_peak(self, df)],
                'baseline': [VisualOutcomes.baseline_vision(self, df)],
                'visits': [len(df)]}
        return pd.DataFrame(data)

    
    def master_dataframe(self):
        """
        Returns a dataframe of statics for all patients.
        """
        df = self.get_df()
        id_list = df["id"].unique()
        frames = []
        for i in range(len(id_list)):
            try:
                pdf = self.dataframe_gen(df, id_list[i])
                pdf['id'] = i
                frames.append(pdf)
            except:
                i += 1
        master = pd.concat(frames)
        master.reset_index(inplace=True)
        master.drop(columns=['index'], inplace=True)
        return master

In [23]:
dataframe = Dataframe()
df = dataframe.master_dataframe()
df.head()

,mean_vision,time_above_baseline,peak_visual_improvement,overall_visual_change,time_to_peak,baseline,visits,id
0,70.588235,0,0.0,-9.333333,0,76.0,34,0
1,71.687500,54,4.0,-5.666667,328,76.0,18,1
2,73.250000,0,0.0,-26.000000,0,85.0,32,2
3,75.392857,1940,20.0,15.000000,987,65.0,58,3
4,75.600000,50,4.0,-4.000000,363,76.0,17,4


In [24]:
len(df)

181

## Predict patient vision
Goal: predict whether overall visual change (OVC, defined as last vision minus first vision) will be positive or negative for a given patient, using feature-engineered columns that summarise patient clinical history.

First, we need to append a column to the dataframe that will determine whether OVC is positive or negative. To do this, we'll write a function to return a 1 or a 0 based on the value of OVC in that row.

In [29]:
def label_ovc(row): return 1 if row['overall_visual_change'] > 0 else 0

Notice that the parameter being input to `label_ovc` is a Series object named row. Next, we use the `apply` function in Pandas to apply the function. We save the results in a new column in the dataframe.

In [32]:
df['OVC_outcome'] = df.apply(lambda row: label_ovc(row), axis=1)

Let's check that the new column is there.

In [33]:
df.head()

,mean_vision,time_above_baseline,peak_visual_improvement,overall_visual_change,time_to_peak,baseline,visits,id,OVC_outcome
0,70.588235,0,0.0,-9.333333,0,76.0,34,0,0
1,71.687500,54,4.0,-5.666667,328,76.0,18,1,0
2,73.250000,0,0.0,-26.000000,0,85.0,32,2,0
3,75.392857,1940,20.0,15.000000,987,65.0,58,3,1
4,75.600000,50,4.0,-4.000000,363,76.0,17,4,0


We now need to save our features (`X`) and targets (`y`) by extracting them from the dataframe. Note that we need to drop `overall_visual_change` from the features as well as `OVC_outcome`, as we are classifying OVC rather than regressing it.

In [35]:
X = df.drop(columns=['overall_visual_change', 'OVC_outcome'])
y = df.OVC_outcome

Use scikit-learn to get our train-test split.

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

We now create the XGBoost model using `XGBClassifier`. We don't actually need to specify the objective here, but it's good for clarity. If we had more than two classes, we would instead use `multi:softmax`, but we'll just use a regular logistic function here.

In [41]:
xgb = XGBClassifier(booster='gbtree', objective='binary:logistic', max_depth=6,
                   learning_rate=0.1, n_estimators=100, random_state=2, n_jobs=-1)

Call `fit` on the model to train it.

In [42]:
xgb.fit(X_train, y_train)

[00:21:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

Get the predictions and store them. Let's see how accurate we are.

In [45]:
y_pred = xgb.predict(X_test)
score = accuracy_score(y_pred, y_test)
print("Accuracy: {}%".format(np.round(100*score,2)))

Accuracy: 76.09%


We achieve 76% accuracy with no hyperparameter tuning and minimal feature engineering.